In [5]:
import os
import cv2
import pytesseract
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
import joblib
import tensorflow.keras as keras

# Step 1: Load the Text Model
text_model_file = 'text_model.pkl'
text_model = joblib.load(text_model_file)

# Step 2: Load the Nature Model
nature_model_file = 'nature_model300.h5'
nature_model = keras.models.load_model(nature_model_file)

# Load the TfidfVectorizer used during training
vectorizer_file = 'vectorizer.pkl'
vectorizer = joblib.load(vectorizer_file)

# Step 3: Function to extract text from an image
def extract_text_from_image(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray)
    return text

# Step 4: Function to preprocess the text for prediction
def preprocess_text(text, vectorizer):
    text_features = vectorizer.transform([text])
    return text_features

# Step 5: Function to preprocess the image for nature classification
def preprocess_image(image):
    img = cv2.resize(image, (224, 224))  # Resize the image to match CNN model input shape
    img = img.astype('float32') / 255.0
    return img

# Step 6: Function to predict the class labels for text and nature
def predict_text_and_nature_classes(image_path, vectorizer):
    text = extract_text_from_image(image_path)
    text_features = preprocess_text(text, vectorizer)
    text_class_label = text_model.predict(text_features)[0]

    img = cv2.imread(image_path)
    nature_class_probabilities = nature_model.predict(np.expand_dims(preprocess_image(img), axis=0))
    nature_class_label = np.argmax(nature_class_probabilities, axis=-1)[0]

    return text_class_label, nature_class_label

# Class names for text and nature classes (replace with your actual class names)
text_class_names = ['Non islamophobic text', 'Islamophobic text']
nature_class_names = ['Islamophobic image', 'Non Islamophobic image']

# Function to get the class names for text and nature classes
def get_class_name(class_label, class_names):
    return class_names[class_label]

# ... (previous code)

# Example usage for video prediction
video_path = 'video.mp4'  # Replace with the actual path to your video file

cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_path = 'frame_temp.jpg'
    cv2.imwrite(frame_path, frame)
    
    if os.path.exists(frame_path):  # Check if the temporary image file exists
        frame_text_class_label, frame_nature_class_label = predict_text_and_nature_classes(frame_path, vectorizer)
        
        if 0 <= frame_text_class_label < len(text_class_names):
            frame_text_class_name = text_class_names[frame_text_class_label]
        else:
            frame_text_class_name = "Unknown Text Class"
        
        if 0 <= frame_nature_class_label < len(nature_class_names):
            frame_nature_class_name = nature_class_names[frame_nature_class_label]
        else:
            frame_nature_class_name = "Unknown Nature Class"
        
        cv2.putText(frame, "Text Class: " + frame_text_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, "Nature Class: " + frame_nature_class_name, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        cv2.imshow('Frame', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        os.remove(frame_path)  # Remove the temporary frame image
    else:
        print("Temporary frame image not found.")
        break

cap.release()
cv2.destroyAllWindows()



1/1 [==============================] - 0s 60ms/step
